In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
8,Wisconsin reports record number of Covid-19 ho...,https://www.cnn.com/2020/09/30/health/us-coron...,"Thu, 01 Oct 2020 01:15:00 GMT",2020-10-01 01:15:00+00:00
7,COVID-19 and African rheumatology: progress in...,https://www.thelancet.com/journals/lanrhe/arti...,"Wed, 30 Sep 2020 22:45:31 GMT",2020-09-30 22:45:31+00:00
1,'Provocative results' boost hopes of antibody ...,https://www.sciencemag.org/news/2020/09/provoc...,"Wed, 30 Sep 2020 21:15:00 GMT",2020-09-30 21:15:00+00:00
5,Sewing Masks — Resilience in the Face of Covid...,https://www.nejm.org/doi/full/10.1056/NEJMp201...,"Wed, 30 Sep 2020 21:01:36 GMT",2020-09-30 21:01:36+00:00
4,Rethinking Covid-19 Test Sensitivity — A Strat...,https://www.nejm.org/doi/full/10.1056/NEJMp202...,"Wed, 30 Sep 2020 21:00:36 GMT",2020-09-30 21:00:36+00:00
0,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Wed, 30 Sep 2020 21:00:00 GMT",2020-09-30 21:00:00+00:00
3,COVID-19 Daily Update 9-30-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Wed, 30 Sep 2020 14:09:13 GMT",2020-09-30 14:09:13+00:00
9,"Alexa, do I have COVID-19? Nature.com",https://www.nature.com/articles/d41586-020-027...,"Wed, 30 Sep 2020 09:03:21 GMT",2020-09-30 09:03:21+00:00
2,NIH to assess and expand COVID-19 testing for ...,https://www.nih.gov/news-events/news-releases/...,"Tue, 29 Sep 2020 19:00:33 GMT",2020-09-29 19:00:33+00:00
6,How stigmatizing disease—from COVID-19 to HIV—...,https://www.sciencemag.org/news/2020/09/how-st...,"Tue, 29 Sep 2020 16:00:00 GMT",2020-09-29 16:00:00+00:00
